In [61]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
data = web.DataReader('2558.JP', 'stooq', start, end)  #MAXIS 米国株式(S&P500)上場投信
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               Open     High      Low    Close   Volume
Date                                                   
2022-10-17  15440.0  15495.0  15385.0  15490.0  48699.0
2022-10-14  15570.0  15710.0  15525.0  15680.0  75963.0
2022-10-13  15170.0  15195.0  15135.0  15145.0  34460.0
2022-10-12  15110.0  15220.0  15090.0  15210.0  35254.0
2022-10-11  15205.0  15215.0  15080.0  15090.0  94267.0
...             ...      ...      ...      ...      ...
2020-01-16  10400.0  10400.0  10320.0  10350.0   1391.0
2020-01-15  10400.0  10400.0  10300.0  10320.0   5119.0
2020-01-14  10390.0  10400.0  10350.0  10370.0   5739.0
2020-01-10  10300.0  10320.0  10280.0  10320.0   2502.0
2020-01-09  10180.0  10250.0  10180.0  10230.0   3389.0

[676 rows x 5 columns]


**Methods**

In [62]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**RSI**

In [63]:
import BacktestingStrategies.Prod_Strategy_Rsi as btstRsi
importlib.reload(btstRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),
    "atr_window": range(10, 30, 2),
    "atr_stoploss": range(1, 10, 1),
    "atr_takeprofit": range(1, 10, 1),
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

stop loss  2020-02-28 00:00:00  10029.401  9147.610837761693
take profit  2020-05-28 00:00:00  7868.7585  9295.65886277087
take profit  2020-08-25 00:00:00  9526.926  10392.104158625569
take profit  2020-12-02 00:00:00  9798.2625  10899.311197389463
take profit  2021-08-31 00:00:00  13406.033  14218.901683499153
take profit  2021-10-26 00:00:00  13677.3695  14859.06420804497
stop loss  2022-01-27 00:00:00  15084.299500000001  14225.07304787732
take profit  2022-03-29 00:00:00  14003.97825  16162.853249289408
take profit  2022-08-19 00:00:00  15069.22525  16593.89630423357
Start                     2020-01-09 00:00:00
End                       2022-10-17 00:00:00
Duration                   1012 days 00:00:00
Exposure Time [%]                   52.810651
Equity Final [$]                  1913699.707
Equity Peak [$]                   1961669.707
Return [%]                          91.369971
Buy & Hold Return [%]                 51.4174
Return (Ann.) [%]                   27.372951
Volatil

C:\Users\javie\AppData\Local\Temp\ipykernel_21936\2036696897.py:6: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,99,32,34,10029.40100,9100.0,-92010.69900,-0.092668,2020-02-27,2020-03-02,4 days
1,115,57,93,7868.75850,9310.0,165742.77250,0.183160,2020-04-03,2020-05-29,56 days
2,112,105,153,9526.92600,10500.0,108984.28800,0.102139,2020-06-16,2020-08-26,71 days
3,120,172,219,9798.26250,11000.0,144208.50000,0.122648,2020-09-24,2020-12-03,70 days
4,98,353,401,13406.03300,14300.0,87608.76600,0.066684,2021-06-22,2021-09-01,71 days
5,103,415,439,13677.36950,15010.0,137260.94150,0.097433,2021-09-22,2021-10-27,35 days
6,102,466,501,15084.29950,14365.0,-73368.54900,-0.047685,2021-12-07,2022-01-28,52 days
7,105,518,541,14003.97825,16380.0,249482.28375,0.169668,2022-02-24,2022-03-30,34 days
8,114,567,638,15069.22525,16570.0,171088.32150,0.099592,2022-05-11,2022-08-22,103 days
9,123,662,675,15320.46275,15440.0,14703.08175,0.007802,2022-09-27,2022-10-17,20 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 988 configurations.
  output = _optimize_grid()


take profit  2020-04-17 00:00:00  7285.8875  8648.033667666781
take profit  2020-05-19 00:00:00  7285.8875  8986.146030692878
stop loss  2020-03-13 00:00:00  9145.045  7680.4100174748155
stop loss  2020-03-09 00:00:00  9145.045  8911.880555456404
stop loss  2020-03-23 00:00:00  7376.333  7077.634964654491
take profit  2020-06-04 00:00:00  8160.194  9659.744877133728
take profit  2020-04-09 00:00:00  7135.145  8336.848831350382
take profit  2020-03-25 00:00:00  7285.8875  7691.725589085918


c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


take profit  2020-05-19 00:00:00  7285.8875  8986.146030692878
take profit  2020-04-10 00:00:00  7285.8875  8555.237268656152
take profit  2020-05-11 00:00:00  7285.8875  8940.31063495028
take profit  2020-04-15 00:00:00  7285.8875  8639.38476855733
stop loss  2020-03-19 00:00:00  9145.045  7444.423333514019
take profit  2022-03-23 00:00:00  14436.10675  15516.914048284285
take profit  2020-08-24 00:00:00  9145.045  10245.503153416974
take profit  2022-04-20 00:00:00  14436.10675  16434.994654702296
stop loss  2020-03-09 00:00:00  9145.045  8918.809112749885
stop loss  2020-03-12 00:00:00  8240.59  7988.441176715379
take profit  2020-04-28 00:00:00  7376.333  8733.93133398564
take profit  2020-04-28 00:00:00  7376.333  8733.93133398564
take profit  2020-05-26 00:00:00  7376.333  9186.810423594594
stop loss  2020-03-16 00:00:00  8160.194  7795.128169560037
stop loss  2020-03-19 00:00:00  7728.0655  7356.141436779665
take profit  2020-03-27 00:00:00  7506.9765  7906.604338745806
take pro

c:\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


stop loss  2020-03-23 00:00:00  9145.045  7185.099345935032
take profit  2020-11-24 00:00:00  9828.411  10737.91535884701
take profit  2022-03-28 00:00:00  14330.587  15938.51625264938
stop loss  2020-03-23 00:00:00  9145.045  7095.478333091876
take profit  2022-02-02 00:00:00  14436.10675  14893.858648639496
stop loss  2020-03-23 00:00:00  9145.045  7229.977553174284
take profit  2022-02-01 00:00:00  14436.10675  14854.757248648093
stop loss  2020-03-11 00:00:00  9145.045  8399.13770189461
stop loss  2022-02-24 00:00:00  14436.10675  13736.8594182513
stop loss  2020-03-09 00:00:00  9145.045  8602.406252695142
stop loss  2020-03-18 00:00:00  8240.59  7579.150235370644
take profit  2020-08-06 00:00:00  9145.045  10003.618947434852
take profit  2020-08-13 00:00:00  9145.045  10240.508461697145
stop loss  2020-03-09 00:00:00  9145.045  8895.407058538964
stop loss  2020-03-12 00:00:00  8240.59  7961.333146518533
take profit  2020-05-20 00:00:00  7376.333  9000.325372524574
take profit  202

KeyboardInterrupt: 